In [732]:
import sys
import string 
import time
import timeit
import re
import nltk
from nltk.stem import *
from nltk.corpus import stopwords
from nltk.stem.porter import *
from collections import Counter

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

stemmer = PorterStemmer()
dict_stop = stopwords.words('english')

In [738]:
def processing(line):
        line = line.lower().strip()
        line = re.sub(r'<(.*?)>',' ',line)
        line = re.sub(r'[\w\d]{20,}','',line)
        line = re.sub(r'[\d]','',line)
        line = re.sub(r'[.]{1}','',line)
        line = re.sub(r'- the new york times','',line)
        line = line.translate(dict.fromkeys(map(ord,punc)))
        line = line.replace('"','').replace('“','').replace('”','').replace('’','').replace('•','').replace('‘','').replace('—','')
        words = [word for word in nltk.word_tokenize(line) if word not in dict_stop]
        line = ' '.join(list(map(stemmer.stem,words)))
        if len(line) != 0:
            return(line.strip())

In [ ]:
# Insert directory:
dirname = '/Users/mueric35/Box Sync/nytimes_articles'

for filename in os.listdir(dirname):
    
    print("Loading: %s" % filename,file=sys.stderr)
    
    ### Read Data as lines
    lines = open(dirname + '/' + filename).readlines()
    
    ### Words Processing Pipeline
    punc = nltk.word_tokenize(string.punctuation)
    punc.remove('*+')
    punc = ''.join(punc)
    
        
    print('Data Tidying', file = sys.stderr)
    start = timeit.default_timer()
    L = list(map(processing,lines))
    L = [l for l in L if l != None]

    elapsed = timeit.default_timer() - start
    print('time elapsed ' + str(elapsed), file = sys.stderr)

    print('Export Text list', file = sys.stderr)
    text = []
    index = [index for index, value in enumerate(L) if value == '************']
    j = 0
    for i in index:
        text.append(' '.join(L[j:i-1]))
        j = i+1

    f = open('tidy_' + filename,'w')
    for i in text:
        f.write(i+'\n')
    f.close()



Loading: article_201607.txt
Data Tidying
time elapsed101.80138798803091
Export Text list
Loading: article_201608.txt
Data Tidying
time elapsed101.34319578006398
Export Text list
Loading: article_201609.txt
Data Tidying
time elapsed110.61778589803725
Export Text list
Loading: article_201610.txt
Data Tidying
time elapsed115.92811990401242
Export Text list
Loading: article_201611.txt
Data Tidying
time elapsed109.2160108149983
Export Text list
Loading: article_201612.txt
Data Tidying
time elapsed95.2971899540862
Export Text list
Loading: article_201701.txt
Data Tidying
time elapsed97.66063083801419
Export Text list
Loading: article_201702.txt
Data Tidying
time elapsed92.41644197702408
Export Text list
Loading: article_201703.txt
Data Tidying
time elapsed112.39821139001288
Export Text list
Loading: article_201704.txt
Data Tidying


### Bigram

In [681]:
start = timeit.default_timer()
text_bigram = list(map(lambda x: list(nltk.bigrams(nltk.word_tokenize(x))),text))
bigrm = [bi for Bi in text_bigram for bi in Bi]
elapsed = timeit.default_timer() - start
print(elapsed)

10.455914280959405


### Count the bigram

In [683]:
bigrm_table = Counter(bigrm)

In [684]:
bigrm_table.most_common()

[(('new', 'york'), 3917),
 (('mr', 'trump'), 3410),
 (('unit', 'state'), 3091),
 (('last', 'year'), 1426),
 (('presid', 'trump'), 1313),
 (('white', 'hous'), 1146),
 (('york', 'time'), 1123),
 (('said', 'mr'), 959),
 (('mr', 'comey'), 940),
 (('health', 'care'), 847),
 (('year', 'ago'), 788),
 (('last', 'week'), 683),
 (('islam', 'state'), 673),
 (('chief', 'execut'), 672),
 (('york', 'citi'), 598),
 (('lo', 'angel'), 595),
 (('last', 'month'), 558),
 (('climat', 'chang'), 538),
 (('prime', 'minist'), 535),
 (('trump', 'administr'), 530),
 (('first', 'time'), 465),
 (('attorney', 'gener'), 460),
 (('north', 'korea'), 456),
 (('social', 'media'), 451),
 (('high', 'school'), 449),
 (('two', 'year'), 447),
 (('suprem', 'court'), 447),
 (('said', 'would'), 433),
 (('offici', 'said'), 403),
 (('polic', 'offic'), 383),
 (('look', 'like'), 383),
 (('mr', 'cosbi'), 375),
 (('european', 'union'), 372),
 (('said', 'statement'), 365),
 (('mr', 'may'), 363),
 (('saudi', 'arabia'), 362),
 (('nation